# 🤖 ComfyUI GPU Colab - 100% AUTOMATED

## 🎯 Zero Manual Intervention

Este notebook é **100% automatizado** e se comunica diretamente com GitHub Actions.

### 📋 Como Funciona:

1. **GitHub Actions inicia este notebook** (via trigger)
2. **Notebook instala e inicia ComfyUI** automaticamente
3. **Cloudflare Tunnel cria URL pública** automaticamente
4. **Script captura URL e envia para Gist** automaticamente
5. **GitHub Actions detecta URL e continua pipeline** automaticamente

**Resultado:** Zero intervenção manual! 🚀

---

In [ ]:
#@title 🔧 Install ComfyUI
import os
import subprocess

print("📦 Installing ComfyUI...")

# Clone ComfyUI
if not os.path.exists('ComfyUI'):
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    %cd ComfyUI
    !pip install -r requirements.txt
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    %cd ..

print("✅ ComfyUI installed!")

In [ ]:
#@title 🚀 Start ComfyUI in Background
import subprocess
import time

print("🚀 Starting ComfyUI...")

# Start ComfyUI in background
comfyui_proc = subprocess.Popen(
    ['python', 'ComfyUI/main.py', '--listen', '--port', '8188'],
    stdout=open('/content/comfyui.log', 'w'),
    stderr=subprocess.STDOUT,
    cwd='/content'
)

print("⏳ Waiting for ComfyUI to start...")
time.sleep(15)

print("✅ ComfyUI started!")
print(f"PID: {comfyui_proc.pid}")

In [ ]:
#@title 📡 Install and Start Cloudflare Tunnel
import subprocess
import time

print("📡 Installing Cloudflare Tunnel...")

!curl -L --output cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared.deb

print("🌐 Starting Cloudflare Tunnel...")

# Start cloudflared in background
cloudflared_proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:8188', '--no-autoupdate'],
    stdout=open('/content/cloudflared.log', 'w'),
    stderr=subprocess.STDOUT,
    cwd='/content'
)

print("⏳ Waiting for tunnel to establish...")
time.sleep(20)

print("✅ Cloudflare Tunnel started!")
print(f"PID: {cloudflared_proc.pid}")

In [ ]:
#@title 📝 HISTÓRIA PARA O FILME
# ═══════════════════════════════════════════════════════════════════
# COLE SUA HISTÓRIA AQUI:
# ═══════════════════════════════════════════════════════════════════

STORY = """
Era uma vez, em um reino distante, uma princesa que adorava matemática.
Ela passava seus dias resolvendo equações e criando teoremas.
Um dia, descobriu um padrão mágico nos números que mudaria tudo...
"""

print("="*60)
print("📖 HISTÓRIA CARREGADA")
print("="*60)
print(f"Caracteres: {len(STORY)}")
print(f"Palavras: {len(STORY.split())}")
print(f"Linhas: {len(STORY.splitlines())}")
print("="*60)
print()

# Salvar em variável de ambiente
import os
os.environ['STORY'] = STORY.strip()


In [ ]:
#@title 🤖 AUTO-REPORTER: Send URL + STORY to GitHub
import re
import time
import json
import requests
import os
from datetime import datetime

# ============================================================
# CONFIGURAÇÃO AUTOMÁTICA VIA SECRETS
# ============================================================
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN', '')
GIST_ID = os.getenv('GIST_ID', '')
STORY = os.getenv('STORY', '')

print("="*60)
print("🤖 AUTO-REPORTER INICIADO")
print("100% Automatizado - Com História")
print("="*60)
print(f"📖 História: {len(STORY)} caracteres")
print()

def capture_url():
    """Captura URL do Cloudflare automaticamente"""
    print("🔍 Capturando URL do Cloudflare...")
    
    for attempt in range(30):
        try:
            with open('/content/cloudflared.log', 'r') as f:
                log_content = f.read()
            
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', log_content)
            
            if match:
                url = match.group(0)
                print(f"✅ URL capturada: {url}")
                return url
            else:
                print(f"⏳ Tentativa {attempt + 1}/30: Aguardando URL...")
                time.sleep(10)
        except:
            print(f"⏳ Tentativa {attempt + 1}/30: Log ainda não disponível...")
            time.sleep(10)
    
    raise Exception("❌ Não foi possível capturar URL")

def verify_comfyui(url):
    """Verifica se ComfyUI está acessível"""
    print(f"🧪 Verificando ComfyUI em {url}...")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            print("✅ ComfyUI está acessível!")
            return True
    except:
        pass
    print("⚠️ ComfyUI ainda não está acessível")
    return False

def report_to_github(url):
    """Reporta URL para GitHub Gist automaticamente"""
    print("📡 Reportando URL para GitHub...")
    
    if not GITHUB_TOKEN:
        print("⚠️ GITHUB_TOKEN não configurado - modo manual")
        print(f"🔗 URL: {url}")
        return False
    
    headers = {
        'Authorization': f'token {GITHUB_TOKEN}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    gist_data = {
        "description": "ComfyUI URL - Auto-reported from Colab",
        "public": False,
        "files": {
            "comfyui_url.json": {
                "content": json.dumps({
                    "url": url,
                    "updated_at": datetime.now().isoformat(),
                    "status": "active",
                    "source": "colab_automated",
                    "verified": True
                }, indent=2)
            }
        }
    }
    
    try:
        if GIST_ID:
            # Atualizar Gist existente
            response = requests.patch(
                f'https://api.github.com/gists/{GIST_ID}',
                headers=headers,
                json=gist_data,
                timeout=30
            )
        else:
            # Criar novo Gist
            response = requests.post(
                'https://api.github.com/gists',
                headers=headers,
                json=gist_data,
                timeout=30
            )
        
        if response.status_code in [200, 201]:
            gist_id = response.json()['id']
            print(f"✅ URL reportada com sucesso!")
            print(f"🔗 Gist ID: {gist_id}")
            return True
        else:
            print(f"❌ Erro: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Erro ao reportar: {e}")
        return False

# ============================================================
# EXECUÇÃO AUTOMÁTICA
# ============================================================
try:
    # 1. Capturar URL
    url = capture_url()
    
    # 2. Verificar ComfyUI
    verify_comfyui(url)
    
    # 3. Reportar para GitHub
    if report_to_github(url):
        print()
        print("="*60)
        print("✅ SUCESSO! URL REPORTADA AUTOMATICAMENTE")
        print(f"🌐 URL: {url}")
        print("🤖 GitHub Actions continuará o pipeline automaticamente")
        print("="*60)
    else:
        print()
        print("="*60)
        print("⚠️ Modo Manual - Configure GITHUB_TOKEN")
        print(f"🔗 URL: {url}")
        print("="*60)
except Exception as e:
    print()
    print("="*60)
    print(f"❌ ERRO: {e}")
    print("="*60)

## 🎉 Sistema Ativo!

### ✅ O que está rodando:

- **ComfyUI:** Gerando imagens com GPU T4
- **Cloudflare Tunnel:** URL pública ativa
- **Auto-Reporter:** URL enviada para GitHub

### 🤖 Próximos Passos Automáticos:

1. GitHub Actions detecta URL no Gist
2. Pipeline continua automaticamente
3. Testes de integração executados
4. Geração de conteúdo iniciada
5. Deploy no Kubernetes (se em main)

**Tudo 100% automatizado!** 🚀

---

### 📊 Monitoramento:

```python
# Ver logs do ComfyUI
!tail -20 /content/comfyui.log

# Ver logs do Cloudflare
!tail -20 /content/cloudflared.log

# Ver processos rodando
!ps aux | grep -E 'python|cloudflared'
```

### 🔄 Manter Colab Ativo:

Este notebook ficará ativo por ~12 horas (limite gratuito do Colab).
Após isso, o GitHub Actions pode reiniciar automaticamente se configurado.